## Libraries

In [1]:
from pathlib import Path
import pandas as pd
import json
import numpy as np


In [2]:
import chromadb
from chromadb.config import Settings


In [3]:
from chromadb import Client


## Filepath

In [4]:
# Get the current working directory
cwd = Path.cwd()

# Get the parent of the working directory
parent_dir = cwd.parent

# Get the parent of the parent directory
grandparent_dir = parent_dir.parent

# Get path to /data/interim
data_interim_dir = grandparent_dir / Path('data') / Path('interim')

## Functions

In [5]:
def get_verses_per_book(data: dict, book_number: int) -> dict:
    # Get the name of the book from the dictionary
    book_text = data[book_number]['name']
    reference_list = [] # keys of output dictionary
    verses_list = [] # values of output dictionary
    # Iterate over all chapters in the dictionary
    for chapter_number, chapter in enumerate(data[book_number]['chapters']):
        # Get the chapter number as a string
        chapter_number_txt = str(chapter_number+1)
        # Iterate over all verses in the chapter using enumerate() function
        for verse_number, verse in enumerate(chapter):
            # Get the verse number as a string
            verse_number_text = str(verse_number+1)
            # Append book name, chapter number and verse number into a string
            reference_list.append(f"{book_text} {chapter_number_txt}:{verse_number_text}")
            # Append verse to list
            verses_list.append(verse)   
    return  dict(zip(reference_list, verses_list))

In [6]:
def get_verses_of_all_books(data: dict) -> list:
    bible_books = []
    for book_number, books in enumerate(data):
        book_verses = get_verses_per_book(data,book_number)
        bible_books.append(book_verses)
    return bible_books

## Execute

In [7]:
filepath = data_interim_dir / Path('en_bbe.json')

with open(filepath) as f:
    data = json.load(f)

In [8]:
bible_books= get_verses_of_all_books(data)

In [9]:
df_bible = pd.DataFrame.from_dict(bible_books).T

In [10]:
df_bible['verse'] = df_bible.apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [11]:
df_bible_flat = df_bible.drop([col for col in df_bible.columns if col != 'verse'], axis=1)

### Test

In [12]:
book_number = 0
data[book_number]
print("Book type")
print(type(data[book_number]))
book_chapters = data[book_number]['chapters']
print("Book chapter type")
print(type(book_chapters))
print("Book chapters in book")
print(len(book_chapters))

Book type
<class 'dict'>
Book chapter type
<class 'list'>
Book chapters in book
50


In [13]:
len(book_chapters[1])

25

In [14]:
df_bible.iat[0,0]

'At the first God made the heaven and the earth.'

In [15]:
df_bible_flat.iat[0,0]

'At the first God made the heaven and the earth.'

### ChromaDB Setup

In [16]:
filepath = str(data_interim_dir / Path('ChromaDB'))

chroma_client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
                                    persist_directory=filepath
                                ))

Using embedded DuckDB with persistence: data will be stored in: c:\Users\Admin\Documents\Github\scripture-semantic-search\data\interim\ChromaDB


In [17]:
chroma_client.list_collections()

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
c:\Users\Admin\Documents\Github\scripture-semantic-search\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=bible), Collection(name=3john), Collection(name=genesis)]

### Create Embeddings

#### Bible

In [ ]:
collection = chroma_client.get_or_create_collection(name="bible")

"""
collection.add(
        documents = list(df_bible_flat['verse']),
        ids = list(df_bible_flat.index)
        )

# Only have to persist once
chroma_client.persist()
"""


In [ ]:
# collection.peek()
collection.count()

In [ ]:
query = ["Create"]

result = collection.query(
    query_texts=query,
    n_results=7)

In [ ]:
result_df = pd.DataFrame.from_dict(result)
result_df = result_df.drop(['embeddings','metadatas'],axis=1)
result_df = result_df.explode(list(result_df.columns))
result_df.sort_values("distances", ascending=True)

In [ ]:
for i in range(len(result_df)):
    reference = result_df.iat[i,0]
    verse = result_df.iat[i,1]
    print(f"{reference} - {verse}")

### Update Embeddings

If an id is not found in the collection, an exception will be raised. If documents are supplied without corresponding embeddings, the embeddings will be recomupted with the collection's embedding function.

#### Single Book - Single Chapter

In [ ]:
book_number = 63
book_name = data[book_number]['name'].lower().replace(" ", "")
book_chapters = data[book_number]['chapters']

df_book = df_bible.drop([col for col in df_bible.columns if col != book_number], axis=1).dropna().rename(columns = {book_number:'verse'})

In [ ]:
collection = chroma_client.get_or_create_collection(name=book_name)

result = collection.get(
    ids=list(df_book.index),
    include=["embeddings","documents","metadatas"]
)

result_df = pd.DataFrame.from_dict(result)

In [ ]:
book_embeddings = list(result_df.embeddings)
book_documents = list(df_book['verse'])

meta_dict = {"book":book_name, "chapter":str(len(book_chapters))}
book_chapter_verses = len(book_chapters[0])

book_chapter_metadata = [meta_dict] * book_chapter_verses

In [ ]:
print(len(book_embeddings))
print(len(book_documents))
print(len(book_chapter_metadata))

In [ ]:
collection.update(
    ids=list(df_book.index),
    embeddings=book_embeddings,
    documents =book_documents,
    metadatas=book_chapter_metadata,
)

chroma_client.persist()

In [ ]:
result = collection.get(
    ids=list(df_book.index),
    include=["embeddings","documents","metadatas"]
)

pd.DataFrame.from_dict(result)

#### Single Book - Multiple Chapters

In [ ]:
book_number = 0
book_name = data[book_number]['name'].lower().replace(" ", "")
book_chapters = data[book_number]['chapters']

df_book = df_bible.drop([col for col in df_bible.columns if col != book_number], axis=1).dropna().rename(columns = {book_number:'verse'})

In [ ]:
collection = chroma_client.get_or_create_collection(name=book_name)

result = collection.get(
    ids=list(df_book.index),
    include=["embeddings","documents","metadatas"]
)

result_df = pd.DataFrame.from_dict(result)
result_df.head()

In [ ]:
book_embeddings = list(result_df.embeddings)
book_documents = list(df_book['verse'])
book_ids = list(df_book.index)

In [ ]:
book_meta_dict_list = []
for i in range(len(book_chapters)):
    meta_dict = {"book":book_name, "chapter":str(i+1)}
    book_chapter_verses = len(book_chapters[i])
    book_meta_dict_list.append([meta_dict] * book_chapter_verses)

book_meta_dict_list = [item for sublist in book_meta_dict_list for item in sublist]


In [ ]:
print(len(book_embeddings))
print(len(book_documents))
print(len(book_meta_dict_list))

In [ ]:
collection.update(
    ids=book_ids,
    embeddings=book_embeddings,
    documents =book_documents,
    metadatas=book_meta_dict_list,
)

chroma_client.persist()

In [ ]:
result = collection.get(
    ids=list(df_book.index),
    include=["embeddings","documents","metadatas"]
)

result_df = pd.DataFrame.from_dict(result)
result_df.head()

#### Bible

In [ ]:
collection = chroma_client.get_or_create_collection(name='bible')

result = collection.get(
    ids=list(df_bible_flat.index),
    include=["embeddings","documents","metadatas"]
)

result_df = pd.DataFrame.from_dict(result)

In [ ]:
bible_embeddings = list(result_df.embeddings)
bible_documents = list(df_bible_flat['verse'])
bible_ids = list(df_bible_flat.index)

In [ ]:
bible_meta_dict_list = []

for x in range(len(data)):
    book_chapters = data[x]['chapters']
    book_name = data[x]['name'].lower().replace(" ", "")
    book_meta_dict_list = []
    for i in range(len(book_chapters)):
        meta_dict = {"book":book_name, "chapter":str(i+1)}
        book_chapter_verses = len(book_chapters[i])
        book_meta_dict_list.append([meta_dict] * book_chapter_verses)
    book_meta_dict_list = [item for sublist in book_meta_dict_list for item in sublist]
    bible_meta_dict_list.append(book_meta_dict_list)

bible_meta_dict_list = [item for sublist in bible_meta_dict_list for item in sublist]

In [ ]:
print(len(bible_embeddings))
print(len(bible_documents))
print(len(bible_meta_dict_list))

In [ ]:
collection.update(
    ids=bible_ids,
    embeddings=bible_embeddings,
    documents =bible_documents,
    metadatas=bible_meta_dict_list,
)

chroma_client.persist()

In [ ]:
result = collection.get(
    ids=bible_ids,
    include=["embeddings","documents","metadatas"]
)

result_df = pd.DataFrame.from_dict(result)
result_df

### Recalculate Embeddings

#### Bible - Specific Book

In [18]:
book_number = 43
book_name = data[book_number]['name'].lower().replace(" ", "")
print(book_name)

acts


In [19]:
book_chapters = data[book_number]['chapters']

df_book = df_bible.drop([col for col in df_bible.columns if col != book_number], axis=1).dropna().rename(columns = {book_number:'verse'})

In [20]:
bible_book_ids = list(df_book.index)
bible_book_documents = list(df_book['verse'])

In [21]:
bible_book_meta_dict_list = []
for i in range(len(book_chapters)):
    meta_dict = {"book":book_name, "chapter":str(i+1)}
    book_chapter_verses = len(book_chapters[i])
    bible_book_meta_dict_list.append([meta_dict] * book_chapter_verses)

bible_book_meta_dict_list = [item for sublist in bible_book_meta_dict_list for item in sublist]

In [22]:
collection = chroma_client.get_or_create_collection(name='bible')

No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [23]:
collection.update(
    ids=bible_book_ids,
    documents =bible_book_documents,
    metadatas=bible_book_meta_dict_list,
)

chroma_client.persist()